The Vaccine Adverse Event Reporting System (VAERS) is a national early warning system to detect possible safety problems in U.S.-licensed vaccines. 
https://vaers.hhs.gov/about.html

Over 245 million doses of COVID-19 vaccines were administered in the United States from December 14, 2020, through May 3, 2021. During this time, VAERS received 4,178 reports of death (0.0017%) among people who received a COVID-19 vaccine. 
https://www.cdc.gov/coronavirus/2019-ncov/vaccines/safety/adverse-events.html

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data=pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSDATA.csv')
symptom=pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSSYMPTOMS.csv')
vax=pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSVAX.csv')

In [ ]:
data[0:2].T

In [ ]:
symptom[0:2].T

In [ ]:
vax[0:2].T

In [ ]:
vax['VAX_MANU'].value_counts()

In [ ]:
swords=[]
for i in range(len(symptom)):
    swords+=[symptom.iloc[i,1]]
    swords+=[symptom.iloc[i,3]]   
    swords+=[symptom.iloc[i,5]]
    swords+=[symptom.iloc[i,7]]
    swords+=[symptom.iloc[i,9]]

In [ ]:
swords2=list(set(swords)) 
print(len(swords2))
print(swords2[1:100])

In [ ]:
focus0=pd.merge(data,symptom,on='VAERS_ID')
focus1=pd.merge(focus0,vax,on='VAERS_ID')
focus1.columns

# Post-dose Symptoms

In [ ]:
pfizer=focus1[focus1['VAX_MANU']=='PFIZER\BIONTECH']
pfizers=[]
for i in range(len(pfizer)):
    pfizers+=[pfizer.iloc[i,35]]
    pfizers+=[pfizer.iloc[i,37]]   
    pfizers+=[pfizer.iloc[i,39]]
    pfizers+=[pfizer.iloc[i,41]]
    pfizers+=[pfizer.iloc[i,43]]
    
pfizers_df=pd.DataFrame(pfizers)
print(pfizers_df.value_counts()[0:30])
print(len(pfizer))

In [ ]:
moderna=focus1[focus1['VAX_MANU']=='MODERNA']
modernas=[]
for i in range(len(moderna)):
    modernas+=[moderna.iloc[i,35]]
    modernas+=[moderna.iloc[i,37]]   
    modernas+=[moderna.iloc[i,39]]
    modernas+=[moderna.iloc[i,41]]
    modernas+=[moderna.iloc[i,43]]
    
modernas_df=pd.DataFrame(modernas)
print(modernas_df.value_counts()[0:30])
print(len(moderna))

# Case Fatality Ratios by Age Band 

In [ ]:
age_band=[]
for item in focus1['AGE_YRS']:
    if math.isnan(item):
        age_band+=[-1]
    else:
        age_band+=[math.floor(item/10)]
focus1['AGE_BAND']=age_band

In [ ]:
MORT0=[]
for i in range(11):
    total=focus1[focus1['AGE_BAND']==i].shape[0]
    died=focus1[focus1['AGE_BAND']==i][focus1['DIED']=='Y'].shape[0]
    MORT0+=[[i,total,died,died*100/total]]       
mort_df0=pd.DataFrame(MORT0)
mort_df0.columns=['AGE_BAND','TOTAL','DEATH','% RATIO']
mort_df0

In [ ]:
plt.style.use('ggplot') 
font = {'family':'meiryo'}
matplotlib.rc('font', **font)

In [ ]:
print(mort_df0.shape)
print(mort_df0[0:3])

In [ ]:
mort_df0.plot.bar(y=['% RATIO'], alpha=0.8, figsize=(12,6))
plt.title('Case Fatality Ratios by Age Band', size=24)
plt.ylabel('% ratio')

Case Fatality Ratios increase with age. Aged people have higher risk of fatality for vaccination?

In [ ]:
# AGEBAND 0: AGE 0-9
# AGEBAND 1: AGE 10-19
# AGEBAND 2: AGE 20-29
# AGEBAND 3: AGE 30-39
# AGEBAND 4: AGE 40-49
# AGEBAND 5: AGE 50-59
# AGEBAND 6: AGE 60-69
# AGEBAND 7: AGE 70-79
# AGEBAND 8: AGE 80-89
# AGEBAND 9: AGE 90-99
# AGEBAND 10: AGE 100-109  

In [ ]:
MORT1=focus1[['SEX','DIED','AGE_BAND']].copy()
print(MORT1)
MORT2=MORT1[MORT1['DIED']=='Y'].groupby(['AGE_BAND']).sum()

In [ ]:
died_m=[]
died_f=[]
for item in MORT2['SEX']:
    died_m+=[item.count('M')]
    died_f+=[item.count('F')]
MORT2['DIED_MALE']=died_m
MORT2['DIED_FEMALE']=died_f
MORT3=MORT2.drop(['SEX','DIED'],axis=1)
MORT3

In [ ]:
MORT3.plot.bar(y=['DIED_MALE','DIED_FEMALE'], alpha=0.8, figsize=(12,6))
plt.title('Case Fatality Number by Age Band and Sex', size=24)
plt.ylabel('Number')

# History Comparison between Alive and Dead

In [ ]:
Y_histories=focus1[focus1['DIED']=='Y'][focus1['AGE_YRS']>59]['HISTORY']
N_histories=focus1[focus1['DIED']!='Y'][focus1['AGE_YRS']>59]['HISTORY']
print(len(Y_histories))
print(len(N_histories))

In [ ]:
item2=[]
for item in Y_histories:
    if type(item)==str:
        item2+=item.split(', ')

(pd.DataFrame(item2)).value_counts()[0:30]

In [ ]:
item3=[]
for item in N_histories:
    if type(item)==str:
        item3+=item.split(', ')

(pd.DataFrame(item3)).value_counts()[0:30]

"No medical history" seems to be very rare in dead cases compared to alive cases. Risk factors for covid19 could be risks also for vaccination. For high risk people for covid19, we might have better have alternative vaccination methods that are mild for their body.

# Days after vaccination in case fatality

In [ ]:
focus2=focus1[['VAX_DATE','DATEDIED','DIED']].copy()
focus2['VAX_DATE2']=pd.to_datetime(focus2['VAX_DATE'])
focus2['DATEDIED2']=pd.to_datetime(focus2['DATEDIED'])
focus2['DAYS']=focus2['DATEDIED2']-focus2['VAX_DATE2']
days_df0=focus2[focus2['DIED']=='Y']['DAYS'].value_counts()
print(focus2[focus2['DIED']=='Y']['DAYS'].value_counts()[0:5])

In [ ]:
days_df0=pd.DataFrame(days_df0)
days_df0[1]=days_df0.index.days
days_df0.columns=['n','days']
days_df1=days_df0[days_df0['days']<=14][0<=days_df0['days']]
days_df2=days_df1.sort_values(['days'])
days_df3=days_df2.set_index('days')

In [ ]:
days_df3.plot.bar(y=['n'], alpha=0.8, figsize=(12,6))
plt.title('Days after vaccination in case fatality', size=24)
plt.ylabel('cases')

Number of fetal cases was high during 10 days after vaccination, which might indicate that these fetalities were related to vaccination. What happened in their body?